# Model 1

XGB Boost Model

In [ ]:
! pip install -r requirements.txt

In [ ]:
#
# basics
import pandas as pd
import numpy as np

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_palette('magma')
from sklearn.model_selection import train_test_split
import missingno as msno
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_error


# warnings
import warnings
warnings.filterwarnings('ignore')

RSEED = 42

# EDA

In [ ]:
# load training data
df = pd.read_csv('data/Train.csv')

In [ ]:
import matplotlib.pyplot as plt
def plot_nas(df: pd.DataFrame):
    if df.isnull().sum().sum() != 0:
        na_df = (df.isnull().sum() / len(df)) * 100      
        na_df = na_df.drop(na_df[na_df == 0].index).sort_values(ascending=False)
        missing_data = pd.DataFrame({'Missing Ratio %' :na_df})
        missing_data.plot(kind = "barh")
        plt.show()
    else:
        print('No NAs found')
plot_nas(df)
plot_width, plot_height = (15,15)
plt.rcParams['figure.figsize'] = (plot_width,plot_height)
plt.rcParams['ytick.labelsize'] = 9

In [ ]:
# drop columns with more than 50% missing values
check_ch4 = [col for col in df.columns if 'CH4' in col]
df.drop(columns=check_ch4, axis=1, inplace=True)

# Convert to datetime
df["Date"] = pd.to_datetime(df["Date"])

# Create integer column for weekends
df["IsWeek"] = (df["Date"].dt.weekday < 5).astype(int)

df.drop(columns=['Date', 'Place_ID', 'Place_ID X Date', 'target_min', 'target_max', 'target_variance', 'target_count'], axis=1, inplace=True)

df.head(5)

In [ ]:
msno.matrix(df.query('IsWeek == 1'), fontsize=10,labels=True)

In [ ]:
msno.matrix(df.query('IsWeek == 0'), fontsize=10,labels=True)

In [ ]:
#removes 0 as outliers
variables_list_0_as_outlier = [
    "L3_NO2_NO2_column_number_density",
    "L3_NO2_NO2_slant_column_number_density",
    "L3_NO2_sensor_altitude",
    "L3_NO2_tropopause_pressure",
    "L3_NO2_solar_zenith_angle",
    "L3_SO2_SO2_column_number_density",
    "L3_SO2_SO2_column_number_density_amf",
    "L3_SO2_SO2_slant_column_number_density",
    "L3_SO2_solar_zenith_angle",
    "L3_O3_O3_column_number_density",
    "L3_O3_O3_effective_temperature",
    "L3_O3_solar_zenith_angle",
    "L3_CO_CO_column_number_density",
    "L3_CO_sensor_altitude",
    "L3_CO_solar_zenith_angle",
    "L3_HCHO_solar_zenith_angle",
    "L3_HCHO_tropospheric_HCHO_column_number_density_amf"
]

df[variables_list_0_as_outlier].replace(0, np.nan, inplace=True)

In [ ]:
msno.matrix(df, fontsize=10,labels=True)

In [ ]:
X = df.drop("target", axis=1)
y = df.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RSEED)

## Pipeline

In [ ]:
# Pipeline XGBoost Regression

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import xgboost as xgb

features = list(X.columns)

num_pipeline = Pipeline([
    ('imputer_num', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, features),
])

pipe_xgbreg = Pipeline([
    ('preprocessor', preprocessor),
    ('xgbreg', xgb.XGBRegressor(eta = 0.1, n_estimators=50,max_depth=10, colsample_bytree=0.8,gamma=5,reg_alpha=0,reg_lambda=10 )) 
])

pipe_xgbreg

  

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


pipe_xgbreg.fit(X_train, y_train)

y_pred_test = pipe_xgbreg.predict(X_test)
y_pred_train = pipe_xgbreg.predict(X_train)

mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = mse_test ** 0.5
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

mse = mean_squared_error(y_train, y_pred_train)
rmse = mse ** 0.5
mae = mean_absolute_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)
              
            
print(f"TEST: RMSE: {rmse_test:.2f}, MAE: {mae_test:.2f}, R²: {r2_test:.3f}")
print(f"train: RMSE: {rmse:.2f}, MAE: {mae:.2f}, R²: {r2:.3f}")

## RandomSearch

In [ ]:
# param_xgbreg = {
#               'xgbreg__n_estimators': [30],
#               'xgbreg__learning_rate': [0.08, 0.1, 0.14], 
#               'xgbreg__max_depth': [ 7, 8,],
#               'xgbreg__colsample_bytree': [0.7, 0.8, 1],
#               'xgbreg__gamma': [0.2, 0.3, 0.5],
#               'xgbreg__reg_alpha': [0],
#               'xgbreg__reg_lambda': [5, 10]
#             }

# random_search_xbreg = RandomizedSearchCV(pipe_xgbreg, param_distributions=param_xgbreg, cv=5, scoring='neg_mean_absolute_error', 
#                            verbose=1, n_jobs=-1, n_iter=50)

# random_search_xbreg.fit(X_train, y_train)
# random_search_xbreg.best_params_, random_search_xbreg.best_score_


In [ ]:
# best_model = random_search_xbreg.best_estimator_
# y_test_predicted = best_model.predict(X_test)
# y_train_predicted = best_model.predict(X_train)

# print("mean_absolute_error: {:.2f}".format(mean_absolute_error(y_test, y_test_predicted)))
# print("mean_absolute_error_train: {:.2f}".format(mean_absolute_error(y_train, y_train_predicted)))



# GRIDSEARCH Finetuning

In [ ]:
# GRIDSEARCH

# param_xgbreg_grid = {
#               'xgbreg__n_estimators': [50],
#               'xgbreg__learning_rate': [0.1], 
#               'xgbreg__max_depth': [10],
#               'xgbreg__colsample_bytree': [ 0.8],
#               'xgbreg__gamma': [5],
#               'xgbreg__reg_alpha': [0],
#               'xgbreg__reg_lambda': [10]
#             }

# grid_search_xbreg = GridSearchCV(pipe_xgbreg, param_grid=param_xgbreg_grid, cv=5, scoring='neg_mean_absolute_error', 
#                            verbose=1, n_jobs=-1)

# grid_search_xbreg.fit(X_train, y_train)
# grid_search_xbreg.best_params_, grid_search_xbreg.best_score_

In [ ]:
# best_model = grid_search_xbreg.best_estimator_
# y_test_predicted_grid = best_model.predict(X_test)
# y_train_predicted_grid = best_model.predict(X_train)

# print("mean_absolute_error: {:.2f}".format(mean_absolute_error(y_test, y_test_predicted_grid)))
# print("mean_absolute_error_train: {:.2f}".format(mean_absolute_error(y_train, y_train_predicted_grid)))